In [5]:
import numpy as np
import operator

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

Importing data by-line:

In [6]:
with open('coursera_sessions_train.txt') as file:
    train_data = [i.replace('\n', '').split(';') for i in file.readlines()]
train_data[0:4]

[['0,1,2,3,4,5', ''],
 ['9,10,11,9,11,12,9,11', ''],
 ['16,17,18,19,20,21', ''],
 ['24,25,26,27,24', '']]

Creating bag of words from train data.

Count unique indeces and create a dictionary [item]=frequency

In [7]:
bagow_views = []
for i in train_data:
    z = i[0].split(',')
    for j in z:
        bagow_views.append(j)
bagow_purchases = []
for i in train_data:
    z = i[1].split(',')
    for j in z:
        bagow_purchases.append(j)

In [8]:
counts_views = np.unique(bagow_views, return_counts=True)
dictfreq_views = {}
for i, j in zip(counts_views[0], counts_views[1]):
    dictfreq_views[i] = j
counts_purchases = np.unique(bagow_purchases, return_counts=True)
dictfreq_purchases = {}
for i, j in zip(counts_purchases[0], counts_purchases[1]):
    dictfreq_purchases[i] = j

Define function sorts a single line in massive:

In [9]:
def string_sorter(string, dict):
    import operator
    """Sort indeces of items watched by customer popularity-wise
    """
    z = []
    for i in string:
        try:
            z.append((i, dict[i]))
        except KeyError:
            z.append((i, 0))
    return [i[0] for i in sorted(z, key=operator.itemgetter(1), reverse=True)]

In [110]:
def string_sorter_tester(string_sorter):
    base = {'1':1, '2':2, '3':3, '4':4}
    string = (['1']+['2']+['3']+['4'], ['6']+['2']+['8'])
    ans1 = ['4', '3', '2', '1']
    if string_sorter(string[0], base) != ans1:
        print('test 1 error')
    ans2 = ['2', '6', '8']
    if string_sorter(string[1], base) != ans2:
        print('test 2 error')
    base = {'1':1, '2':2, '3':2, '4':4}
    string = (['1']+['3']+['2']+['4']+['6']+['5'], ['6']+['2']+['8'])
    ans3 = ['4', '3', '2', '1', '6', '5']
    if string_sorter(string[0], base) != ans3:
        print('test 3 error')

Get sorted by-line massive where each line contains tuple representing one session and consisted of two lists: first with indices of watched items and latter with indices of items that have been bought.

В итоге будем получать отсортированный массив, где каждая строка это тьюпл, представляющий одну сессию и включающий в свою очередь два листа: первый с индексами просмотренных товаров, второй с индексами купленных.

Define function sorting whole massive:

In [12]:
def massive_sorter(data, dict):
    ''' Sorts indices in massive by-line according to theirs frequencies. Massive must consist
    of lines with a tuple each. Each tuple must contain two lists, both with items indices.
    '''
    sorted = []
    for i in data:
        if len(i[1]) > 1:
            sorted.append((string_sorter(i[0], dict), string_sorter(i[1], dict)))
        else:
               sorted.append((string_sorter(i[0], dict), ''))
    return sorted

Process train data. Making each line in massive a tuple of 2 lists: viewed items indices and purchased items indices. Then soring.

In [120]:
train_data_prepared = [(set(i[0].split(',')), i[1].split(',')) for i in train_data if len(i[1])>0]

In [62]:
train_sorted_views = massive_sorter(train_data_prepared, dictfreq_views)
train_sorted_purchases = massive_sorter(train_data_prepared, dictfreq_purchases)

In [82]:
# Delete sessions without purchases.
train_sorted_views_with_purchases = [i for i in train_sorted_views if len(i[1]) > 0]
train_sorted_purchases_with_purchases = [i for i in train_sorted_purchases if len(i[1]) > 0]

Import and process test data.

In [63]:
with open('coursera_sessions_test.txt') as file:
    test_data = [i.replace('\n', '').split(';') for i in file.readlines()]
test_data[0:4]

[['6,7,8', ''], ['13,14,15', ''], ['22,23', ''], ['28,29,30,31,32,33', '']]

In [64]:
test_data_prepared = [(set(i[0].split(',')), i[1].split(',')) for i in test_data]

In [65]:
test_sorted_views = massive_sorter(test_data_prepared, dictfreq_views)
test_sorted_purchases = massive_sorter(test_data_prepared, dictfreq_purchases)

In [ ]:
# Delete sessions without purchases.
test_sorted_views_with_purchases = [i for i in test_sorted_views if len(i[1]) > 0]
test_sorted_purchases_with_purchases = [i for i in test_sorted_purchases if len(i[1]) > 0]

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Metrics. Average precision and average recall are used.

Define function counts presicion for one session and average precision for whole massive.

In [68]:
def presicionatk(line, km):
    '''Count presicion for a line
    '''
    l = [1 if i in line[1] else 0 for i in line[0]]
    return sum(l[0:km])/km
def presicion(massive, km):
    '''Count average presicion for a massive
    '''
    l = []
    for line in massive:
        l.append(presicionatk(line, km))
    return sum(l)/len(l)

In [69]:
k = 1

p2 = presicion(train_sorted_purchases_with_purchases, k)
print('presicion of recomendations on most purchased items on train sample:', p2)
p3 = presicion(test_sorted_views_with_purchases, k)
print('presicion of recomendations on most viewed items on test sample:', p3)
p4 = presicion(test_sorted_purchases_with_purchases, k)
print('presicion of recomendations on most purchased items on test sample:', p4)

presicion of recomendations on most purchased items on train sample: 0.7467672413793104
presicion of recomendations on most viewed items on test sample: 0.42780172413793105
presicion of recomendations on most purchased items on test sample: 0.4385775862068966


In [70]:
k = 5
p5 = presicion(train_sorted_views_with_purchases, k)
print('presicion of recomendations on most viewed items on train sample:', p1)
p6 = presicion(train_sorted_purchases_with_purchases, k)
print('presicion of recomendations on most purchased items on train sample:', p2)
p7 = presicion(test_sorted_views_with_purchases, k)
print('presicion of recomendations on most viewed items on test sample:', p3)
p8 = presicion(test_sorted_purchases_with_purchases, k)
print('presicion of recomendations on most purchased items on test sample:', p4)

presicion of recomendations on most viewed items on train sample: 0.7489224137931034
presicion of recomendations on most purchased items on train sample: 0.7467672413793104
presicion of recomendations on most viewed items on test sample: 0.42780172413793105
presicion of recomendations on most purchased items on test sample: 0.4385775862068966


In [71]:
def recallatk(line, km):
    '''Count presicion for a line
    '''
    l = [1 if i in line[1] else 0 for i in line[0]]
    return sum(l[0:km])/len(line[1])
def recall(massive, km):
    '''Count average presicion for a massive
    '''
    l = []
    for line in massive:
        l.append(recallatk(line, km))
    return sum(l)/len(l)

In [72]:
def recalltest(recall):
    x1 = [([1, 2, 3], [0, 2]), ([4, 5, 6], [1])]
    if recall(x1, 3) != 0.25:
        print('test1 error')
        return recall(x1, 3)
    x2 = [([1, 2, 3], [0, 2]), ([4, 5], [1, 2, 3])]
    if recall(x2, 3) != 0.25:
        print('test2 error')

In [73]:
recalltest(recall)

In [94]:
# train sample. Recommendations by views rating
k = 1
r1 = recall(train_sorted_views_with_purchases, k)
print('recall on most viewed items on train sample k=1:', r1)
p1 = presicion(train_sorted_views_with_purchases, k)
print('presicion on most viewed items on train sample k=1:', p1)
k = 5
r5 = recall(train_sorted_views_with_purchases, k)
print('recall on most viewed items on train sample k=5:', r5)
p5 = presicion(train_sorted_views_with_purchases, k)
print('presicion on most viewed items on train sample k=5:', p5)
with open('quality_views_train.txt', 'w') as f:
    f.write((' ').join(list(map(lambda x: str(round(x, 2)), [r1, p1, r5, p5]))))

recall on most viewed items on train sample k=1: 0.18165367917523093
presicion on most viewed items on train sample k=1: 0.44935344827586204
recall on most viewed items on train sample k=5: 0.5918377210672469
presicion on most viewed items on train sample k=5: 0.30323275862068866


In [91]:
# train sample. Recommendations by purchasing rating
k = 1
r1 = recall(train_sorted_purchases_with_purchases, k)
print('recall on most purchased items on train sample k=1:', r1)
p1 = presicion(train_sorted_purchases_with_purchases, k)
print('presicion on most purchased items on train sample k=1:', p1)
k = 5
r5 = recall(train_sorted_purchases_with_purchases, k)
print('recall on most purchased items on train sample k=5:', r5)
p5 = presicion(train_sorted_purchases_with_purchases, k)
print('presicion on most purchased items on train sample k=5:', p5)
with open('quality_purchases_train.txt', 'w') as f:
    f.write((' ').join(list(map(lambda x: str(round(x, 2)), [r1, p1, r5, p5]))))

recall on most purchased items on train sample k=1: 0.2991836587741761
presicion on most purchased items on train sample k=1: 0.7467672413793104
recall on most purchased items on train sample k=5: 0.7792672425754316
presicion on most purchased items on train sample k=5: 0.4165948275862045


In [92]:
# test sample. Recommendations by views rating
k = 1
r1 = recall(test_sorted_views_with_purchases, k)
print('recall on most viewed items on test sample k=1:', r1)
p1 = presicion(test_sorted_views_with_purchases, k)
print('presicion on most viewed items on test sample k=1:', p1)
k = 5
r5 = recall(test_sorted_views_with_purchases, k)
print('recall on most viewed items on test sample k=5:', r5)
p5 = presicion(test_sorted_views_with_purchases, k)
print('presicion on most viewed items on test sample k=5:', p5)
with open('quality_views_test.txt', 'w') as f:
    f.write((' ').join(list(map(lambda x: str(round(x, 2)), [r1, p1, r5, p5]))))

recall on most viewed items on test sample k=1: 0.1756982929666119
presicion on most viewed items on test sample k=1: 0.42780172413793105
recall on most viewed items on test sample k=5: 0.5619503583534137
presicion on most viewed items on test sample k=5: 0.2855603448275858


In [93]:
# test sample. Recommendations by purchasing rating
k = 1
r1 = recall(test_sorted_purchases_with_purchases, k)
print('recall on most purchased items on test sample k=1:', r1)
p1 = presicion(test_sorted_purchases_with_purchases, k)
print('presicion on most purchased items on test sample k=1:', p1)
k = 5
r5 = recall(test_sorted_purchases_with_purchases, k)
print('recall on most purchased items on test sample k=5:', r5)
p5 = presicion(test_sorted_purchases_with_purchases, k)
print('presicion on most purchased items on test sample k=5:', p5)
with open('quality_purchases_test.txt', 'w') as f:
    f.write((' ').join(list(map(lambda x: str(round(x, 2)), [r1, p1, r5, p5]))))

recall on most purchased items on test sample k=1: 0.1815880285303777
presicion on most purchased items on test sample k=1: 0.4385775862068966
recall on most purchased items on test sample k=5: 0.5538708010994777
presicion on most purchased items on test sample k=5: 0.28232758620689624
